In [215]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz
from sklearn.dummy import DummyClassifier

In [216]:
df = pd.read_csv('https://raw.githubusercontent.com/ImagineDogs/TranstornosMentais/main/dados/df_cid10.csv')
table = df[['cid10_faixa', 'cid10_faixa_alta', 'cid10_seg_faixa']].loc[~df['cid10_faixa'].isnull()]
table = table.fillna('Sem')

Para da inicio as predições é necessário utilizar um encoder adequado para os dados, por se tratarem de categorias independentes, ou seja, não possuem ordem, será utilizaro o OneHotEncoder que transforma cada categoria em uma coluna de valores binarios.

In [217]:
encX = OneHotEncoder()
ency = OneHotEncoder()

X = table[['cid10_faixa', 'cid10_seg_faixa']]
X = encX.fit_transform(X)
y = np.array(table['cid10_faixa_alta'])
# y = ency.fit_transform(y).reshape(-1, 1)

Para fazer análises mais profundas de configurações podemos separar nossos dados em  treino, teste e validação, porém neste primeiro momento vamos utilizar apenas treino e teste por se tratar de uma abordagem mais simplista inicialmente. Contúdo uma função para as três separações já pode ser mantida pronta.

In [218]:
def train_test_val_split(X, y, test_size=None, val_size = None,random_state=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=random_state)
    
    return X_train.toarray(), X_test.toarray(), X_val.toarray(), y_train, y_test, y_val

# X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, y, test_size=0.2, val_size = 0.05,random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, = X_train.toarray(), X_test.toarray()

In [219]:
print(X_train.shape)
X_train

(6574, 23)


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [220]:
print(y_train.shape)
y_train

(6574,)


array(['F10-F19', 'F10-F19', 'F20-F29', ..., 'F10-F19', 'F30-F39',
       'F30-F39'], dtype=object)

Serão utilizados modelos explicativos em um primeiro momento para mantermos a explicabilidade das predições.

Os dois modelos escolhidos foram NaiveBayes e DecisionTree por serem algoritmos básicos porém eficazes.

Será utilizado também um modelo baseline para comparação.

In [221]:
# Avaliação dos modelos
def evaluate(y_pred, y_test):
    acuracia = accuracy_score(y_pred, y_test)
    f1 = f1_score(y_pred, y_test, average='weighted')

    print('Resultados:')
    print(f'    Acuracia: {acuracia}')
    print(f'    F1: {f1}')
    return acuracia, f1

In [222]:
dummy = DummyClassifier()
dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)
acuracia_dummy, f1_dummy = evaluate(y_pred, y_test)

Resultados:
    Acuracia: 0.37165450121654503
    F1: 0.5419068736141907


In [223]:
nb = CategoricalNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
acuracia_nb, f1_nb = evaluate(y_pred, y_test)

Resultados:
    Acuracia: 0.7390510948905109
    F1: 0.7488247891294786


Para a Árvore de Decisão será utilizado GridSearch para encontrar os melhores parâmetros dentro de um escopo que pode ser escolhido.

Vale lembrar que GridSearch testa cada combinação de parâmetros passados em um K-Fold Cross Validation, utilizando a combinação que melhor desempenhar.
Neste caso serão 5 folds com a métrica F1 balanceada.

In [224]:
tc = DecisionTreeClassifier(random_state=42)
param_tc = {'criterion': ['gini', 'entropy', 'log_loss'], 'splitter': ['best', 'random'], 'max_depth': range(5, 20, 1)}


gs_tc = GridSearchCV(tc, param_tc, cv=5, scoring='f1_weighted')


best_model = gs_tc.fit(X_train, y_train)
print(best_model.best_params_)

c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


{'criterion': 'entropy', 'max_depth': 7, 'splitter': 'best'}


In [225]:
# Gerador de imagem pra a árvore

# Definindo os labels das features
model = DecisionTreeClassifier(random_state=42, criterion=best_model.best_params_['criterion'], max_depth=best_model.best_params_['max_depth'], splitter=best_model.best_params_['splitter'])
model.fit(X_train, y_train)
df2 = pd.DataFrame(encX.inverse_transform(X_train), columns=['cid10_faixa', 'cid10_seg_faixa'])
labels = df2['cid10_faixa'].apply(lambda x: 'cid10_faixa_' + x).unique().tolist()
labels.extend(df2['cid10_seg_faixa'].apply(lambda x: 'cid10_seg_faixa_' + x).unique().tolist())
labels

#Gerando o gráfico
dot_data = export_graphviz(model, out_file=None, 
                           feature_names=labels,  
                           class_names=np.unique(y_train).tolist(),  
                           filled=True, rounded=True,  
                           special_characters=True)  

graph = graphviz.Source(dot_data)
graph.render("iris_tree")

'iris_tree.pdf'

In [226]:
y_pred = best_model.predict(X_test)
acuracia_dt, f1_dt = evaluate(y_pred, y_test)

Resultados:
    Acuracia: 0.7402676399026764
    F1: 0.7511145511008342


In [227]:
resultados = pd.DataFrame()
resultados['NaiveBayes'] = pd.Series([acuracia_nb, f1_nb])
resultados['DecisionTree'] = pd.Series([acuracia_dt, f1_dt])
resultados['Dummy'] = pd.Series([acuracia_dummy, f1_dummy])
resultados.index = ['Acuracia', 'F1 Balanceado']
resultados

,NaiveBayes,DecisionTree,Dummy
Acuracia,0.739051,0.740268,0.371655
F1 Balanceado,0.748825,0.751115,0.541907
